<a href="https://colab.research.google.com/github/salmon-Xin/use-pyserini-build-chinese-wikipedia-inverted-index/blob/main/%E4%B8%AD%E6%96%87%E7%B6%AD%E5%9F%BA%E7%99%BE%E7%A7%91%E5%8F%8D%E5%90%91%E7%B4%A2%E5%BC%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m" -O wiki_2021_08_05_1215639.json && rm -rf /tmp/cookies.txt

--2023-03-16 09:26:12--  https://docs.google.com/uc?export=download&confirm=t&id=1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m
Resolving docs.google.com (docs.google.com)... 172.253.119.113, 172.253.119.101, 172.253.119.102, ...
Connecting to docs.google.com (docs.google.com)|172.253.119.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nqv5332532a4fq14balv9edh2qsvdaj3/1678958700000/02591176256574788800/*/1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m?e=download&uuid=9819c605-bd3f-4226-9aa2-e0a90b8b0566 [following]
--2023-03-16 09:26:12--  https://doc-0s-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nqv5332532a4fq14balv9edh2qsvdaj3/1678958700000/02591176256574788800/*/1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m?e=download&uuid=9819c605-bd3f-4226-9aa2-e0a90b8b0566
Resolving doc-0s-94-docs.googleusercontent.com (doc-0s-94-docs.googleusercontent.com)... 172.217.214.

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1yyqTwl3SCMQ6P6ZWt88cPqvtzK4RdI2m
To: /content/wiki_2021_08_05_1215639.json
100% 2.95G/2.95G [00:14<00:00, 205MB/s]


In [ ]:
!ls

sample_data  wiki_2021_08_05_1215639.json  wiki.json


In [ ]:
import json
from pprint import pprint

with open("./wiki_2021_08_05_1215639.json", "r") as f:
  data =json.load(f)
  print(len(data))
  pprint(data[0])

1215639
{'articles': '福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈。在香港、澳門、臺灣、菲律賓、新加坡、馬來西亞等地區均有業務。福斯家庭電影頻道已簽訂電影合約的單位包括20世紀福斯、迪斯尼、米高梅 '
             '- 梅耶，Studio Cana、獅門娛樂、頂峯娛樂、溫斯坦公司等。該電影頻道面向老人和兒童，主要提供家庭電影。',
 'id': 1,
 'title': '福斯家庭電影頻道'}


In [ ]:
import json

str1=[]
with open("./wiki_2021_08_05_1215639.json", "r") as f:    #開啟檔案
  data =json.load(f)
  id=""
  context=""
  for d in data:
    id = d['id']
    context = d['title'] + d['articles']
    str1.append({"id":id,"contents":context})

with open('./data/data.json', 'w') as f:
  j = json.dump(str1,f, indent=2)


In [ ]:
import json
from pprint import pprint

with open("./data/data.json", "r") as f:
  data =json.load(f)
  print(len(data))
  pprint(data[0])

1215639
{'contents': '福斯家庭電影頻道福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈。在香港、澳門、臺灣、菲律賓、新加坡、馬來西亞等地區均有業務。福斯家庭電影頻道已簽訂電影合約的單位包括20世紀福斯、迪斯尼、米高梅 '
             '- 梅耶，Studio Cana、獅門娛樂、頂峯娛樂、溫斯坦公司等。該電影頻道面向老人和兒童，主要提供家庭電影。',
 'id': 1}


In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-03-16 10:08:45,070 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-16 10:08:45,083 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-16 10:08:45,083 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-16 10:08:45,084 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: data
2023-03-16 10:08:45,084 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-03-16 10:08:45,085 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-16 10:08:45,085 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-03-16 10:08:45,086 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: en
2023-03-16 10:08:45,086 INFO  [main] index.IndexCollection (IndexCollection.j

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('indexes/sample_collection_jsonl')
hits = searcher.search('蝙蝠俠')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 239924 17.89590
 2 71183 17.83050
 3 247385 17.82790
 4 245815 17.77300
 5 231025 17.75640
 6 417600 17.74240
 7 249910 17.70610
 8 1137420 17.66220
 9 1199669 17.64700
10 1208679 17.64430


In [ ]:
index_reader.stats()

{'total_terms': 437792340,
 'documents': 1215639,
 'non_empty_documents': 1215639,
 'unique_terms': 863549}

In [ ]:
from pyserini.index.lucene import IndexReader
import itertools

# Initialize from an index path:
index_reader = IndexReader('indexes/sample_collection_jsonl/')

for term in itertools.islice(index_reader.terms(), 10):
    print(f'{term.term} (df={term.df}, cf={term.cf})')
    

0 (df=23227, cf=54012)
0'0 (df=1, cf=1)
0'18 (df=1, cf=1)
0'2 (df=1, cf=1)
0'21 (df=1, cf=2)
0'22 (df=1, cf=1)
0'26 (df=1, cf=1)
0'31 (df=2, cf=2)
0'36 (df=1, cf=1)
0'37 (df=1, cf=1)


In [ ]:
term = '蝙蝠俠'

# Look up its document frequency (df) and collection frequency (cf).
# Note, we use the unanalyzed form:
df, cf = index_reader.get_term_counts(term)
print(f'term "{term}": df={df}, cf={cf}')


term "蝙蝠俠": df=534, cf=None


In [ ]:
import math

idf = math.log(10,(index_reader.stats()['documents']/df))
print(f'term "{term}": df={df}, idf={idf}')

term "蝙蝠俠": df=534, idf=0.29786164816652444


**"蝙蝠俠" 一詞**

**df = 534**

**idf = 0.29786164816652444**


In [ ]:
!python -m pyserini.search.lucene \
  --index indexes/sample_collection_jsonl \
  --topics sample_queries.tsv \
  --output run.sample.txt \
  --bm25

Running sample_queries.tsv topics, saving to run.sample.txt...
100% 1/1 [00:01<00:00,  1.97s/it]


In [ ]:
!cat run.sample.txt

1 Q0 239924 1 17.895901 Anserini
1 Q0 71183 2 17.830500 Anserini
1 Q0 247385 3 17.827900 Anserini
1 Q0 245815 4 17.773001 Anserini
1 Q0 231025 5 17.756399 Anserini
1 Q0 417600 6 17.742399 Anserini
1 Q0 249910 7 17.706100 Anserini
1 Q0 1137420 8 17.662201 Anserini
1 Q0 1199669 9 17.646999 Anserini
1 Q0 1208679 10 17.644300 Anserini
1 Q0 199211 11 17.642900 Anserini
1 Q0 475839 12 17.627399 Anserini
1 Q0 488525 13 17.600800 Anserini
1 Q0 1137603 14 17.585600 Anserini
1 Q0 103910 15 17.552299 Anserini
1 Q0 1045511 16 17.540300 Anserini
1 Q0 339944 17 17.533600 Anserini
1 Q0 250593 18 17.513399 Anserini
1 Q0 298377 19 17.499100 Anserini
1 Q0 107075 20 17.482401 Anserini
1 Q0 158156 21 17.449800 Anserini
1 Q0 1167598 22 17.409401 Anserini
1 Q0 587367 23 17.404699 Anserini
1 Q0 710072 24 17.388700 Anserini
1 Q0 943026 25 17.371201 Anserini
1 Q0 113906 26 17.351200 Anserini
1 Q0 912782 27 17.349001 Anserini
1 Q0 752899 28 17.345800 Anserini
1 Q0 228336 29 17.339199 Anserini
1 Q0 36268 30 17.3

In [ ]:
# Fetch and traverse postings for an unanalyzed term:
postings_list = index_reader.get_postings_list(term)
for posting in postings_list:
    print(f'docid={posting.docid}, tf={posting.tf}, pos={posting.positions}')

docid=353, tf=4, pos=[29, 38, 47, 197]
docid=526, tf=2, pos=[33, 57]
docid=955, tf=2, pos=[35, 111]
docid=2148, tf=12, pos=[1, 11, 44, 128, 207, 369, 390, 418, 519, 563, 589, 613]
docid=2368, tf=1, pos=[177]
docid=2638, tf=5, pos=[1954, 1995, 2205, 2303, 2328]
docid=3216, tf=2, pos=[3019, 3034]
docid=3584, tf=1, pos=[2321]
docid=3875, tf=1, pos=[1735]
docid=5361, tf=3, pos=[2, 45, 266]
docid=6580, tf=2, pos=[102, 106]
docid=6605, tf=1, pos=[1470]
docid=7169, tf=1, pos=[39]
docid=8505, tf=1, pos=[120]
docid=9183, tf=1, pos=[259]
docid=10736, tf=1, pos=[65]
docid=11668, tf=3, pos=[130, 150, 189]
docid=13287, tf=8, pos=[0, 13, 60, 87, 92, 177, 206, 421]
docid=14363, tf=20, pos=[0, 8, 134, 150, 160, 205, 250, 277, 434, 575, 623, 648, 836, 855, 883, 1068, 1093, 1128, 1173, 1219]
docid=14824, tf=3, pos=[42, 59, 97]
docid=15102, tf=9, pos=[2, 10, 53, 72, 95, 192, 230, 268, 277]
docid=18140, tf=3, pos=[9186, 9587, 10385]
docid=18491, tf=4, pos=[120, 158, 364, 403]
docid=18750, tf=26, pos=[453,